In [97]:
import pandas as pd
import numpy as np
import os 
#import torch
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors  # K-Nearest Neighbors
from sklearn.linear_model import LogisticRegression  # Logistic Regression
from sklearn.tree import DecisionTreeClassifier  # Decision Tree
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# from scraping import get_ingredients
from SMILES import CIRconvert
import pubchempy as pcp
import cirpy
import ast
import time
import csv
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
# import faiss

In [98]:
df = pd.read_csv(r'C:\Users\tiffa\Documents\HackUMass25\skincare_products_clean.csv')
df

,product_name,product_url,product_type,clean_ingreds,price
0,The Ordinary Natural Moisturising Factors + HA...,https://www.lookfantastic.com/the-ordinary-nat...,Moisturiser,"['capric triglyceride', 'cetyl alcohol', 'prop...",£5.20
1,CeraVe Facial Moisturising Lotion SPF 25 52ml,https://www.lookfantastic.com/cerave-facial-mo...,Moisturiser,"['homosalate', 'glycerin', 'octocrylene', 'eth...",£13.00
2,The Ordinary Hyaluronic Acid 2% + B5 Hydration...,https://www.lookfantastic.com/the-ordinary-hya...,Moisturiser,"['sodium hyaluronate', 'sodium hyaluronate', '...",£6.20
3,AMELIORATE Transforming Body Lotion 200ml,https://www.lookfantastic.com/ameliorate-trans...,Moisturiser,"['ammonium lactate', 'c12-15', 'glycerin', 'pr...",£22.50
4,CeraVe Moisturising Cream 454g,https://www.lookfantastic.com/cerave-moisturis...,Moisturiser,"['glycerin', 'cetearyl alcohol', 'capric trigl...",£16.00
...,...,...,...,...,...
1133,Elemis Life Elixirs Embrace Bath and Shower El...,https://www.lookfantastic.com/elemis-life-elix...,Bath Oil,"['prunus amygdalus dulcis', 'tipa-laureth sulf...",£55.00
1134,Love Boo Splendidly Soothing Bath Soak (250ml),https://www.lookfantastic.com/love-boo-splendi...,Bath Oil,"['sodium lauroyl', 'sodium cocoamphoacetate', ...",£10.99
1135,Elemis Life Elixirs Fortitude Bath and Shower ...,https://www.lookfantastic.com/elemis-life-elix...,Bath Oil,"['prunus amygdalus dulcis', 'tipa-laureth sulf...",£55.00
1136,Connock London Kukui Oil Soothing Bath & Showe...,https://www.lookfantastic.com/connock-london-k...,Bath Oil,"['capric triglyceride', 'peg-40 sorbitan perol...",£36.00


In [42]:
# Function to split DataFrame into chunks
def split_dataframe(df, chunk_size):
    for i in range(0, df.shape[0], chunk_size):
        yield df.iloc[i:i + chunk_size]

In [ ]:
# # Search through the PubChem database and get the CID of the compound by querying the name
# batches = list(split_dataframe(df, 2))
# for i, batch in enumerate(batches):
#     print(f"Batch {i + 1}:\n{batch}\n")
#     for row_ingredients in batch['clean_ingreds']:
#         print(row_ingredients)
#         row_ingredients = ast.literal_eval(row_ingredients)
#         row_ingredients = [n.strip() for n in row_ingredients]
#         print(type(row_ingredients))
#         for ingredient in row_ingredients: 
#             result_list = []
#             result = cirpy.resolve(f'{ingredient}', 'smiles')
#             if result is None:
#                 continue
#             result_list.append(result)
#         print("result list:", result_list)
#         with open('SMILES.csv', mode='w', newline='') as file:
#             writer = csv.writer(file)
#             writer.writerows(result_list)
#         print(result_list)
#         time.sleep(2)  # Pause for 2 seconds

Batch 1:
                                        product_name  \
0  The Ordinary Natural Moisturising Factors + HA...   
1      CeraVe Facial Moisturising Lotion SPF 25 52ml   

                                         product_url product_type  \
0  https://www.lookfantastic.com/the-ordinary-nat...  Moisturiser   
1  https://www.lookfantastic.com/cerave-facial-mo...  Moisturiser   

                                       clean_ingreds   price  
0  ['capric triglyceride', 'cetyl alcohol', 'prop...   £5.20  
1  ['homosalate', 'glycerin', 'octocrylene', 'eth...  £13.00  

['capric triglyceride', 'cetyl alcohol', 'propanediol', 'stearyl alcohol', 'glycerin', 'sodium hyaluronate', 'arganine', 'aspartic acid', 'glycine', 'alanine', 'serine', 'valine', 'isoleucine', 'proline', 'threonine', 'histidine', 'phenylalanine', 'glucose', 'maltose', 'fructose', 'trehalose', 'sodium pca', 'pca', 'sodium lactate', 'urea', 'allantoin', 'linoleic acid', 'oleic acid', 'phytosteryl canola glycerides', 'palm

HTTPError: HTTP Error 500: INTERNAL SERVER ERROR

In [99]:
smiles_df = pd.read_csv(r'smiles_with_no_nulls.csv')
smiles_df

,Unnamed: 0,product_name,product_type,clean_ingreds,price,SMILES
0,0,The Ordinary Natural Moisturising Factors + HA...,Moisturiser,"['capric triglyceride', 'cetyl alcohol', 'prop...",£5.20,"['CCCCCCCCCCCCCCCCO', 'CCC(O)O', 'CCCCCCCCCCCC..."
1,1,CeraVe Facial Moisturising Lotion SPF 25 52ml,Moisturiser,"['homosalate', 'glycerin', 'octocrylene', 'eth...",£13.00,"['CC1CC(CC(C)(C)C1)OC(=O)c2ccccc2O', 'OCC(O)CO..."
2,2,The Ordinary Hyaluronic Acid 2% + B5 Hydration...,Moisturiser,"['sodium hyaluronate', 'sodium hyaluronate', '...",£6.20,['[Na+].CC(=O)N[C@H]1[C@H](O)O[C@H](CO)[C@@H](...
3,3,AMELIORATE Transforming Body Lotion 200ml,Moisturiser,"['ammonium lactate', 'c12-15', 'glycerin', 'pr...",£22.50,"['[NH4+].CC(O)C([O-])=O', 'OCC(O)CO', 'CCCCCCC..."
4,4,CeraVe Moisturising Cream 454g,Moisturiser,"['glycerin', 'cetearyl alcohol', 'capric trigl...",£16.00,"['OCC(O)CO', 'CCCCCCCCCCCCCCCCO.CCCCCCCCCCCCCC..."
...,...,...,...,...,...,...
1117,1133,Elemis Life Elixirs Embrace Bath and Shower El...,Bath Oil,"['prunus amygdalus dulcis', 'tipa-laureth sulf...",£55.00,"['CC(O)CO', 'CCCCCCCC\\C=C/CCCCCCCC(=O)OCC(O)[..."
1118,1134,Love Boo Splendidly Soothing Bath Soak (250ml),Bath Oil,"['sodium lauroyl', 'sodium cocoamphoacetate', ...",£10.99,['CCCCCCCCCCCCCCCCCCOC(=O)CC(O)(CC([O-])=O)C([...
1119,1135,Elemis Life Elixirs Fortitude Bath and Shower ...,Bath Oil,"['prunus amygdalus dulcis', 'tipa-laureth sulf...",£55.00,"['CC(O)CO', 'CCCCCCCC\\C=C/CCCCCCCC(=O)OCC(O)[..."
1120,1136,Connock London Kukui Oil Soothing Bath & Showe...,Bath Oil,"['capric triglyceride', 'peg-40 sorbitan perol...",£36.00,"['CCCCCCCCCCCCCCCCCCOCC(C)O', 'O=C(OCc1ccccc1)..."


In [100]:
# Generate fingerprints of all the molecules and store them in a df
# all_ingredients_list is the column containing the list of smiles representations of the ingredients of a skincare product
skipped = []
def generate_dataset_fingerprints(df):
    all_fingerprint_list = [] # 2d array that contains the lists of fingerprints of every row
    all_SMILES_list = df['SMILES']
    i = 0
    for smiles_list in all_SMILES_list:
        # make sure that it's a list and not a string
        smiles_list = ast.literal_eval(smiles_list)
        smiles_list = [n.strip() for n in smiles_list]

        # Convert SMILES to RDKit Molecule objects 
        single_fp_list = []
        for r in smiles_list:
            mol = Chem.MolFromSmiles(r)
            try:
                single_fp_list.append(AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048))
            except:
                skipped.append(i)
                continue
        print(single_fp_list)
        i += 1
        all_fingerprint_list.append(single_fp_list)
    return all_fingerprint_list

In [101]:
all_fingerprints = generate_dataset_fingerprints(smiles_df) # 2d array that contains the lists of fingerprints of every row
all_fingerprints

[<rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x0000025E1EC94A30>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x0000025E1EC94DA0>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x0000025E1ED4D3A0>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x0000025E1ED4DB70>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x0000025E1ED4D850>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x0000025E1ED4D8F0>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x0000025E1ED4D6C0>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x0000025E1ED4D170>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x0000025E1ED4D580>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x0000025E1ED4D800>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x0000025E1ED4D210>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x0000025E1ED4D5D0>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect obj

[[<rdkit.DataStructs.cDataStructs.ExplicitBitVect at 0x25e1ec94a30>,
 ...]

In [ ]:
fingerprints_df = pd.DataFrame({
    'Fingerprints': all_fingerprints})
fingerprints_df

,Fingerprints
0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,"[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,"[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
...,...
1117,"[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1118,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,..."
1119,"[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1120,"[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,..."


In [ ]:
# put together smiles and fingerprints dataframes
merged_df = pd.concat([smiles_df, fingerprints_df], axis=1)
merged_df

,Unnamed: 0,product_name,product_type,clean_ingreds,price,SMILES,Fingerprints
0,0,The Ordinary Natural Moisturising Factors + HA...,Moisturiser,"['capric triglyceride', 'cetyl alcohol', 'prop...",£5.20,"['CCCCCCCCCCCCCCCCO', 'CCC(O)O', 'CCCCCCCCCCCC...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,1,CeraVe Facial Moisturising Lotion SPF 25 52ml,Moisturiser,"['homosalate', 'glycerin', 'octocrylene', 'eth...",£13.00,"['CC1CC(CC(C)(C)C1)OC(=O)c2ccccc2O', 'OCC(O)CO...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
2,2,The Ordinary Hyaluronic Acid 2% + B5 Hydration...,Moisturiser,"['sodium hyaluronate', 'sodium hyaluronate', '...",£6.20,['[Na+].CC(=O)N[C@H]1[C@H](O)O[C@H](CO)[C@@H](...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,3,AMELIORATE Transforming Body Lotion 200ml,Moisturiser,"['ammonium lactate', 'c12-15', 'glycerin', 'pr...",£22.50,"['[NH4+].CC(O)C([O-])=O', 'OCC(O)CO', 'CCCCCCC...","[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,4,CeraVe Moisturising Cream 454g,Moisturiser,"['glycerin', 'cetearyl alcohol', 'capric trigl...",£16.00,"['OCC(O)CO', 'CCCCCCCCCCCCCCCCO.CCCCCCCCCCCCCC...","[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
...,...,...,...,...,...,...,...
1117,1133,Elemis Life Elixirs Embrace Bath and Shower El...,Bath Oil,"['prunus amygdalus dulcis', 'tipa-laureth sulf...",£55.00,"['CC(O)CO', 'CCCCCCCC\\C=C/CCCCCCCC(=O)OCC(O)[...","[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1118,1134,Love Boo Splendidly Soothing Bath Soak (250ml),Bath Oil,"['sodium lauroyl', 'sodium cocoamphoacetate', ...",£10.99,['CCCCCCCCCCCCCCCCCCOC(=O)CC(O)(CC([O-])=O)C([...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,..."
1119,1135,Elemis Life Elixirs Fortitude Bath and Shower ...,Bath Oil,"['prunus amygdalus dulcis', 'tipa-laureth sulf...",£55.00,"['CC(O)CO', 'CCCCCCCC\\C=C/CCCCCCCC(=O)OCC(O)[...","[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1120,1136,Connock London Kukui Oil Soothing Bath & Showe...,Bath Oil,"['capric triglyceride', 'peg-40 sorbitan perol...",£36.00,"['CCCCCCCCCCCCCCCCCCOCC(C)O', 'O=C(OCc1ccccc1)...","[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,..."


In [109]:
merged_df.to_csv('fingerprints_and_smiles.csv', index=False)

In [ ]:
# Convert the SMILES to fingerprints and calculates Tanimoto similarity
# Used for model evaluation
def compare_user_similarity(user_ingredients_list):
    # make sure that it's a list
    ingredients = ast.literal_eval(user_ingredients_list)
    ingredients = [n.strip() for n in ingredients]
    result_list = [] # store the smiles representations in a list
    # turn the user's ingredients into smile representations
    for i in ingredients:
        result = cirpy.resolve(f'{i}', 'smiles')
        if result is None:
            continue
        result_list.append(result)

    # Convert SMILES to RDKit Molecule objects 
    fingerprint_list = []
    for r in result_list:
        mol = Chem.MolFromSmiles(r)
        try:
            # Generate Morgan Fingerprints
            fingerprint_list.append(AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048))
        except:
            continue
    
    # Calculate Tanimoto similarity
    # similarity = DataStructs.TanimotoSimilarity(fingerprint1, fingerprint2)